In [7]:
# Imports
import torch
from torch import nn
from src.utils.preprocess_utils import midi_to_note_indices, notes_to_midi
import numpy as np
import torch.nn.functional as F
from mido import MidiFile, MidiTrack, Message
from datetime import datetime

In [8]:
# Define model architecture
class TransformerModel(nn.Module):
    def __init__(self, vocab_size, embedding_size, num_heads, hidden_size, num_layers, dropout=0.2):
        super(TransformerModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_size)
        self.transformer = nn.Transformer(
            d_model=embedding_size,
            nhead=num_heads,
            num_encoder_layers=num_layers,
            dim_feedforward=hidden_size,
            dropout=dropout,
            batch_first=True
        )
        self.fc = nn.Linear(embedding_size, vocab_size)

    def forward(self, x):
        # x: [batch_size, sequence_length]
        x = x.long()
        embedded = self.embedding(x)  # [batch_size, sequence_length, embedding_size]
        
        # Transformer automatically handles positional encodings internally
        transformer_out = self.transformer.encoder(embedded)  # [batch_size, sequence_length, embedding_size]
        
        output = self.fc(transformer_out)  # [batch_size, sequence_length, vocab_size]
        return output

In [9]:
# Load model
model_name = "2025-01-06_18-44-50_e32_nh4_h128_l3_sl25"
model = torch.load(f'weights/model_3/{model_name}')
model.eval()

C:\Users\janikbalint\AppData\Local\Temp\ipykernel_4080\2104852713.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(f'weights/model_3/{model_name}')


TransformerModel(
  (embedding): Embedding(85, 32)
  (transformer): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0-2): 3 x TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=32, out_features=32, bias=True)
          )
          (linear1): Linear(in_features=32, out_features=128, bias=True)
          (dropout): Dropout(p=0.2, inplace=False)
          (linear2): Linear(in_features=128, out_features=32, bias=True)
          (norm1): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.2, inplace=False)
          (dropout2): Dropout(p=0.2, inplace=False)
        )
      )
      (norm): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
    )
    (decoder): TransformerDecoder(
      (layers): ModuleList(
        (0-5): 6 x TransformerDecoderLayer(
          (sel

In [10]:
# Load input
input = midi_to_note_indices("data/input.mid", 24, 108)
sequence_length = 150  # Desired length of the generated sequence
temperature = 0.6
print(input)

[24, 26, 31, 36]


In [11]:
def generate_sequence(model, start_sequence, sequence_length, num_notes, temperature=1.0):
    model.eval()

    start_tensor = torch.tensor(start_sequence, dtype=torch.long).unsqueeze(0)  # [1, sequence_length, input_size]

    generated_sequence = list(start_sequence)

    for i in range(sequence_length):
        with torch.no_grad():
            output = model(start_tensor)  # [1, i, output_size]
            logits = output.squeeze(0)  # Remove the batch dimension: [i, output_size]

            # Temperature scaling
            scaled_logits = logits
            if temperature != 0:
                scaled_logits = logits / temperature

            # Apply softmax to get probabilities for the next note
            output_probs = F.softmax(scaled_logits, dim=-1)

            # Sample the next note based on the probabilities
            next_note = torch.multinomial(output_probs[-1], 1).item()

            # Append the predicted note to the sequence and this sequence's one-hot
            # encoded format will be the input for the next note's generation
            generated_sequence.append(next_note)
            generated_sequence_tensor = np.array(generated_sequence)
            start_tensor = torch.tensor(generated_sequence_tensor, dtype=torch.long).unsqueeze(0)  # Add batch dimension
    
    return generated_sequence

generated_sequence = generate_sequence(model.cpu(), input, sequence_length, num_notes=85, temperature=temperature)

print("Generated Sequence:")
print(generated_sequence)

Generated Sequence:
[24, 26, 31, 36, 67, 68, 48, 43, 75, 73, 4, 11, 19, 7, 36, 62, 45, 57, 15, 3, 67, 79, 67, 64, 15, 3, 39, 31, 19, 63, 64, 62, 61, 54, 51, 76, 16, 50, 84, 38, 75, 63, 14, 8, 23, 73, 12, 0, 12, 68, 66, 78, 41, 59, 14, 81, 14, 78, 12, 64, 22, 25, 29, 78, 35, 80, 34, 47, 8, 14, 63, 8, 20, 75, 4, 34, 71, 36, 15, 8, 17, 66, 40, 12, 40, 5, 57, 82, 56, 52, 79, 20, 8, 15, 3, 53, 77, 29, 14, 83, 25, 65, 19, 58, 29, 48, 50, 17, 54, 20, 5, 72, 8, 36, 61, 54, 57, 8, 14, 11, 59, 20, 8, 4, 19, 50, 18, 32, 40, 6, 29, 33, 5, 54, 17, 1, 2, 82, 22, 61, 24, 21, 49, 24, 12, 66, 54, 17, 5, 27, 62, 28, 56, 61]


In [12]:
current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
file_path = f"generations/model_3/{current_datetime}_t{temperature:.1f}_w{sequence_length}_M_{model_name}.mid"

notes_to_midi(generated_sequence, file_path)

MIDI file saved to generations/model_3/2025-01-06_18-58-03_t0.6_w150_M_2025-01-06_18-44-50_e32_nh4_h128_l3_sl25.mid
